# Pré-Processamento <br>
O pré-processamento dos dados em machine learning é um dos passos fundamentais em um projeto que se propõe a realizar predições de Machine Learning, a famosa frase "garbage in, garbage out" atribuída ao técnico da IBM George Fuechsel que significa, "entra lixo, sai lixo", demonstra muito bem o porque essa fase é tão importante, se não prepararmos os nossos dados da maneira correta os resultados podem ser mentirosos, ou não ter utilidade alguma no final, e, muitas vezes podem passar despercebidos, por esta razão essas é uma das fases mais importantes na hora de manipular dados.<br>

Uma boa parte desse processo já foi realizado pela equipe do Hospital Sírio-Libanês que disponibilizou esses dados [aqui](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19), de acordo com o hospital esse conjunto de dados contém dados anonimizados coletados no Hospital Sírio-Libanês, localizados em São Paulo e em Brasília. Toda a anonimização desses dados seguiu as melhores práticas e recomendações internacionais, e os dados passaram por um processo de limpeza e normalização por coluna de acordo com os valores máximos e mínimos de forma que todos os valores estivessem no intervalo entre -1 e 1.<br>
## Estrutura dos dados
### Chave primária<br>
A coluna 'PATIENT_VISIT_IDENTIFIER' é composta por números inteiros e é responsável por identificar cada paciente diferente.<br>
### Variável a ser prevista<br>
A variável a ser prevista é a coluna 'ICU', que, no conjunto de dados original, indica se o paciente correspondente estava ou não na UTI naquele momento.<br>
### Janela de tempo<br>
De acordo com o Hospital a variável 'WINDOW' diz respeito à janela de tempo onde as medições foram realizadas, ela está organizada da seguinte maneira:<br>

|Janela | Descrição|
|:-------------|:----------:|
|0-2 | Entre 0 até 2 horas a partir da admissão do paciente 
|2-4 | Entre 2 até 4 horas a partir da admissão do paciente 
|4-6 | Entre 4 até 6 horas a partir da admissão do paciente 
|6-12| Entre 6 até 12 horas a partir da admissão do paciente 
|Above-12| Mais de 12 horas horas a partir da admissão do paciente 

### Demais variáveis
As demais colunas do conjunto conjunto de dados trazem informações sobre:
* Informações demográficas do paciente (03)
* Grupos de doenças previamente identificadas pelos pacientes (09)
* Resultados de exames de sangue (36)
* Sinais vitais (06)<br>

No total são 54 variáveis, correspondente às médias, medianas, máximos, minímos, diferenças e diferenças relativas dos dados do paciente.<br>

### Dados faltantes<br>

Um dos maiores desafios ao se analisar dados médicos é a variação entre diferentes tipos de medições, por exemplo, os sinais vitais são coletados com mais frequência (geralmente de hora em hora) do que os laboratórios de sangue (geralmente diariamente).Fato esse acaba causando diversos dados faltantes uma vez que estamos analisando todos esses dados juntos em um mesmo conjunto.<br>

De acordo com o Hospital, para solucionar o problema dos dados faltantes, é razoável supor que um paciente que não tem uma medição registrada em uma janela de tempo esteja clinicamente estável, podendo apresentar sinais vitais e exames de sangue semelhantes às janelas vizinhas. Portanto, pode-se preencher os valores ausentes usando a entrada seguinte ou anterior.<br>  
### Problema a ser resolvido<br>
A identificação precoce dos pacientes que desenvolverão um curso adverso da doença (e precisam de cuidados intensivos) é a chave para um tratamento adequado (salvar vidas) e para gerenciar leitos e recursos. Um bom modelo usando apenas o primeiro (0-2) provavelmente será mais clinicamente relevante, por esta razão os dados serão agrupados de forma reorganizados a fim de agrupar os dados médicos por paciente e apenas as informações da primeira janela serão utilizadas para identificar se um paciente precisou de internação em qualquer uma das janelas.

## Importação das bibliotecas

In [1]:
import pandas as pd
import os

## Importação das funções locais

In [2]:
os.chdir('../funcoes')
from preprocessing import fill_exam, filter_window
os.chdir('../notebooks')

In [3]:
help(fill_exam)

Help on function fill_exam in module preprocessing:

fill_exam(data, group_id='PATIENT_VISIT_IDENTIFIER', target_variable='ICU')
    Função que agrupa os exames por paciente e pela variável independente(ICU), após isso chama a função fill_table para realizar o preenchimento dos valores nulos, dessa forma  
    garantindo que os dados usados para preencher serão somente antes do paciente ter ido para UTI
    
    Parâmetros:
    -----------
    data : conjunto de dados
    group_id : coluna correspondente ao id do paciente, padrão : 'PATIENT_VISIT_IDENTIFIER'
    target_variable : nome da variável dependente, padrão : 'ICU'
    
    Retono:
    -------
    Retorna o conjunto de dados preenchido
    
    OBS: NÃO ESQUECE QUE MUDOU



In [4]:
help(filter_window)

Help on function filter_window in module preprocessing:

filter_window(data, window='0-2', target_variable='ICU', group_id='PATIENT_VISIT_IDENTIFIER')
    Função que agrupa os dados por paciente filtra para janela selecionada e verifica se o paciente foi para UTI em algum momento, e com isso modifica o valor da coluna dependente.Além 
    disso elimina os dados dos pacientes que já estavam na UTI na janela selecionada
    
    Parâmetros:
    -----------
    data : conjunto de dados 
    window : coluna correspondente à janela que será mantida, padrão : '0-2'
    target_variable : nome da variável dependente, padrão : 'ICU'
    group_id : coluna correspondente ao id do paciente, padrão : 'PATIENT_VISIT_IDENTIFIER'
    
    Retorno:
    -------
    Retorna o conjunto de dados na janela definida e com a coluna ICU se esse paciente foi para UTI em algum momento



## Leitura dos dados brutos

Primeiramente os dados brutos serão lidos e armazenados em um DataFrame do pandas.

In [5]:
df = pd.read_excel('../dados/dados_brutos/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx')

# Primeiras impressões

Após os dados estarem armazenados em 

In [6]:
df.shape

(1925, 231)

In [7]:
df.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433,ABOVE_12,1


In [8]:
df = fill_exam(df)

In [9]:
df.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-0.389967,0.407558,-0.230462,0.096774,-1.000000,-1.000000,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-0.389967,0.407558,-0.230462,0.096774,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433,ABOVE_12,1


In [10]:
df = filter_window(df)

In [11]:
df = df.dropna()

In [12]:
df.head(10)

,,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
2,10,2,0,10th,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-0.959596,-0.515528,-0.351328,-0.747001,-0.756272,-1.000000,-0.961262,0-2,1
3,15,3,0,40th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
4,20,4,0,10th,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.952381,-0.979798,-1.000000,-0.883669,-0.956805,-0.870968,-0.953536,-0.980333,0-2,0
5,25,5,0,10th,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.976190,-0.979798,-0.860870,-0.714460,-0.986481,-1.000000,-0.975891,-0.980129,0-2,0
6,30,6,1,70th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
7,35,7,0,20th,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
8,40,8,0,50th,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
9,45,9,1,80th,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0


In [13]:
df.describe()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,...,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU
count,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,...,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000
mean,192.549858,0.458689,0.381766,0.105413,0.022792,0.091168,0.019943,0.128205,0.042735,0.193732,...,-0.961957,-0.965134,-0.985323,-0.962986,-0.958804,-0.976353,-0.964289,-0.965628,-0.985349,0.461538
std,110.807231,0.499002,0.486513,0.307523,0.149453,0.288259,0.140004,0.334795,0.202548,0.395786,...,0.110550,0.086235,0.032677,0.106265,0.117708,0.059678,0.095232,0.084406,0.033063,0.499230
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,97.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
50%,191.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
75%,289.500000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-0.979798,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.980333,1.000000
max,384.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-0.058824,-0.238095,-0.757576,0.130435,0.378657,-0.535878,-0.268817,-0.285468,-0.732438,1.000000


In [14]:
df = df.drop(['WINDOW', 'PATIENT_VISIT_IDENTIFIER'],axis=1)

In [15]:
df.to_csv('../dados/dados_preprocessados/Kaggle_Sirio_Libanes_ICU_Prediction.csv', index=False)